# 벡터 필드 생성
- 참고자료: https://milvus.io/docs/ko/v2.5.x/index-vector-fields.md?tab=floating

## 개요
- Milvus는 인덱스 파일에 저장된 메타데이터를 활용하여 데이터를 특수한 구조로 구성함으로써 검색 또는 쿼리 중에 요청된 정보를 신속하게 검색할 수 있도록 도와줌
- 부동 소수점 임베딩
    - 메트릭 유형
        1. 유클리드 거리(L2)
        2. 내적 곱(IP)
        3. 코사인 유사도(COSINE)
    - 인덱스 유형
        1. FLAT
        2. IVF_FLAT
        3. IVF_SQ8
        4. IVF_PQ
        5. GPU_IVF_FLAT
        6. GPU_IVF_PQ
        7. HNSW
        8. DISKANN

- 자주 액세스하는 벡터 필드와 스칼라 필드 모두에 대한 인덱스를 만드는 것이 좋음

## 준비
- Milvus는 컬렉션을 생성할 때 다음 중 하나라도 지정되면 자동으로 인덱스를 생성하여 메모리에 로드함
    - 벡터 필드의 차원 및 메트릭 유형
    - 스키마 및 인덱스 매개변수

In [2]:
from pymilvus import MilvusClient, DataType

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://localhost:19530"
)

# 2. Create schema
# 2.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)

# 2.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)

# 3. Create collection
client.create_collection(
    collection_name="customized_setup",
    schema=schema
)

## 컬렉션 색인 생성
- 컬렉션의 인덱스를 생성하거나 컬렉션을 색인하려면, 다음을 사용하여 prepare_index_params()를 사용하여 인덱스 파라미터를 준비하고 create_index()를 사용하여 인덱스를 생성

In [4]:
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="vector", # 이 객체를 적용할 대상 파일의 이름
    metric_type="COSINE", # 벡터 간의 유사성을 측정하는 데 사용되는 알고리즘 -> IP, L2, COSINE, JACCARD, HAMMING
    index_type="IVF_FLAT", # -> 특정 필드에서 데이터를 정렬하는 데 사용되는 알고리즘, 인메모리 인덱스 및 온디스크 인덱스를 참조
    index_name="vector_index", # 개체가 적용된 후 생성된 인덱스 파일의 이름
    params={"nlist": 128} 
)

client.create_index(
    collection_name="customized_setup", # 기존 컬렉션 이름
    index_params=index_params, # IndexParams 객체 목록이 포함된 IndexParams 객체
    sync=False # 클라이언트의 요청과 관련하여 인덱스가 빌드되는 방식을 제어 -> True(인덱스가 완전히 빌드될 때까지 기다렸다가 반환), False(클라이언트 요청이 수신되고 인덱스가 백그라운드에서 빌드되는 즉시 반환)
)

## 인덱스 세부 정보 확인
- 인덱스 세부 정보를 확인하려면 list_indexes()을 사용하여 인덱스 이름을 나열하고 describe_index()을 사용하여 인덱스 세부 정보를 가져옴

In [5]:
res = client.list_indexes(
    collection_name="customized_setup"
)

print(res)

['vector_index']


In [6]:
res = client.describe_index(
    collection_name="customized_setup",
    index_name="vector_index"
)

print(res)

{'nlist': '128', 'metric_type': 'COSINE', 'index_type': 'IVF_FLAT', 'field_name': 'vector', 'index_name': 'vector_index', 'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0, 'state': 'Finished'}


## 인덱스 삭제
- 인덱스가 더 이상 필요하지 않은 경우 간단히 삭제 가능

In [7]:
client.drop_index(
    collection_name="customized_setup",
    index_name="vector_index"
)